In [ ]:
# !python -m pip install yahooquery --q

In [1]:
import os
import mlflow
from mlflow.langchain import autolog

from yahooquery import Ticker
import requests

In [2]:
MODEL_NAME = 'gemini-2.0-flash'
COMPANY_NAME = 'Apple Inc.'

In [3]:
# function to generate stock_code from Company Name
company_to_ticker = {
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Amazon": "AMZN",
    "Google": "GOOGL",
    "Tesla": "TSLA"
}

In [4]:

def get_headline_news(stock_code: str) -> str:
    url = f"https://www.alphavantage.co/query"
    params = {
        'function': 'NEWS_SENTIMENT',
        'tickers': stock_code,
        'apikey': 'H1439C91G0VHFCJX',
        'limit': 5
    }
    response = requests.get(url, params=params)
    data = response.json()
    headlines = []
    for article in data['feed'][:5]:
        headlines.append(article['title'])
    print(headlines)
    return {'headlines' :'\n '.join(headlines)}


In [5]:
headlines = get_headline_news('AAPL')

["Meta Unveils $499 Oakley Vanguard And $799 Ray-Ban Display Smart Glasses At Connect 2025: Gene Munster Says 'Best Bang For The Buck' - Meta Platforms  ( NASDAQ:META ) ", 'APPLE INVESTIGATION INITIATED BY FORMER LOUISIANA ATTORNEY GENERAL: Kahn Swick & Foti, LLC Investigates the Officers and Directors of Apple Inc. - AAPL - Apple  ( NASDAQ:AAPL ) ', "Reaction to the Fed - 'Melt Up' or 'Sell the News'; China Bans Nvidia Chips - SPDR S&P 500  ( ARCA:SPY ) ", "Elon Musk's Big India Test: Can Tesla Pull An Apple-Style Breakthrough? - Tesla  ( NASDAQ:TSLA ) ", 'Competitor Analysis: Evaluating Apple And Competitors In Technology Hardware, Storage & Peripherals Industry - Apple  ( NASDAQ:AAPL ) ']


In [6]:
headlines['headlines']

"Meta Unveils $499 Oakley Vanguard And $799 Ray-Ban Display Smart Glasses At Connect 2025: Gene Munster Says 'Best Bang For The Buck' - Meta Platforms  ( NASDAQ:META ) \n APPLE INVESTIGATION INITIATED BY FORMER LOUISIANA ATTORNEY GENERAL: Kahn Swick & Foti, LLC Investigates the Officers and Directors of Apple Inc. - AAPL - Apple  ( NASDAQ:AAPL ) \n Reaction to the Fed - 'Melt Up' or 'Sell the News'; China Bans Nvidia Chips - SPDR S&P 500  ( ARCA:SPY ) \n Elon Musk's Big India Test: Can Tesla Pull An Apple-Style Breakthrough? - Tesla  ( NASDAQ:TSLA ) \n Competitor Analysis: Evaluating Apple And Competitors In Technology Hardware, Storage & Peripherals Industry - Apple  ( NASDAQ:AAPL ) "

In [ ]:

system_prompt = """You are a financial analyst specializing in market sentiment analysis. 
Analyze the following news data about {company_name} (stock code: {stock_code} and provide a structured sentiment analysis.

News Data:
{news_data}

Please provide your analysis in the following JSON format ONLY:
{{
    "company_name": "{company_name}",
    "stock_code": "{stock_code}",
    "newsdesc": "Brief summary of the news (max 200 words)",
    "sentiment": "Positive/Negative/Neutral",
    "people_names": ["list of people mentioned in the news"],
    "places_names": ["list of places/locations mentioned"],
    "other_companies_referred": ["list of other companies mentioned"],
    "related_industries": ["list of related industries mentioned"],
    "market_implications": "Analysis of potential market impact (max 150 words)",
    "confidence_score": 0.0
}}

Guidelines:
- confidence_score should be between 0.0 and 1.0 (1.0 = very confident)
- sentiment should be based on overall tone and potential market impact
- Extract only relevant named entities
- Focus on factual analysis over speculation
- If news data is limited, indicate this in your analysis

Respond with ONLY the JSON object, no additional text."""

from langchain_core.prompts import ChatPromptTemplate
prompt_temp = ChatPromptTemplate.from_messages([
    ("system", system_prompt)
])

from langchain.chat_models import init_chat_model
model = init_chat_model(MODEL_NAME, model_provider="google_genai")

from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()


company_name = 'Apple'
stock_code = company_to_ticker[company_name]
news_data = headlines['headlines']


chain = prompt_temp | model | parser
chain.invoke({"company_name":company_name,"stock_code":stock_code,"news_data":news_data})